In [33]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers


data = pd.read_csv("data.csv", sep=";" )

print(data.columns)

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default payment next month'],
      dtype='object')


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

omit = []
X = data.drop('default payment next month', axis=1)
X = X.drop(omit, axis=1)
y = data['default payment next month']

def deleteFromList(list, omition_list):
    return [item for item in list if item not in omition_list]




# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = pd.concat([X_train, y_train], axis=1)

# Separa las clases
non_default = train_data[train_data['default payment next month'] == 0]
default = train_data[train_data['default payment next month'] == 1]

# Realiza el submuestreo (undersampling)
non_default_undersampled = non_default.sample(len(default), random_state=42)

# Combina las clases de nuevo
undersampled_data = pd.concat([non_default_undersampled, default])

# Separa las características y las etiquetas de nuevo
X_train_undersampled = undersampled_data.drop('default payment next month', axis=1)
y_train_undersampled = undersampled_data['default payment next month']


#X_train = X_train_undersampled
#y_train = y_train_undersampled

# Columnas numéricas y categóricas
num_attribs = deleteFromList(['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], omit)
cat_attribs = deleteFromList(['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], omit)

# Tubos de preprocesamiento
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('encoder', OneHotEncoder()),
    ])

# Combinar todo
preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)


In [35]:
# %%
# Construir el modelo
from tensorflow.keras import layers, regularizers


model = keras.Sequential([
 # Añadir Dropout después de la primera capa oculta con una tasa del 20%
    layers.Dense(64, activation='relu' ),
    layers.Dense(32, activation='relu' ),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)



In [36]:
X_train_full, X_val, y_train_full, y_val = train_test_split(
    X_train_prepared, y_train, test_size=0.2, random_state=42)

X_train_full = X_train_full.toarray()
X_val = X_val.toarray()




history = model.fit(
    X_train_full, y_train_full,
    epochs=50,
    batch_size=100,
    validation_data=(X_val, y_val)
)


Epoch 1/50


192/192 [==============================] - 1s 3ms/step - loss: 0.5138 - accuracy: 0.7595 - val_loss: 0.4440 - val_accuracy: 0.8106
Epoch 2/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4350 - accuracy: 0.8189 - val_loss: 0.4414 - val_accuracy: 0.8146
Epoch 3/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4292 - accuracy: 0.8226 - val_loss: 0.4363 - val_accuracy: 0.8129
Epoch 4/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4255 - accuracy: 0.8235 - val_loss: 0.4403 - val_accuracy: 0.8131
Epoch 5/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4232 - accuracy: 0.8243 - val_loss: 0.4365 - val_accuracy: 0.8138
Epoch 6/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4214 - accuracy: 0.8240 - val_loss: 0.4359 - val_accuracy: 0.8133
Epoch 7/50
192/192 [==============================] - 0s 2ms/step - loss: 0.4201 - accuracy: 0.8243 - val_loss: 0.4404 - val_accuracy: 0.8112
Epoch 8/50
192/19

In [37]:
model.evaluate(X_test_prepared.toarray(), y_test, verbose = 2)

188/188 - 0s - loss: 0.5094 - accuracy: 0.7998 - 332ms/epoch - 2ms/step


[0.5094188451766968, 0.799833357334137]